In [ ]:
https://molecular-cancer.biomedcentral.com/articles/10.1186/s12943-024-02182-w



cool study and we can potentially do something like this:



 determine metastatic genes specific for gex cancer
2. use single cell data to look which cells express these genes
3. use this to predict metastatic features/cell type locations on WSI
4. use spatial to for validation

## Actions
We will try to find metastatic-related genes for LUAD/LUSC by doing the following:

1. Short-list prognostic genes by univariate analysis (p < 0.05)
2. Cross-validate with Lasso-Cox model
3. Further short-list genes by multivariate analysis (p < 0.05)

## Load & format data

In [15]:
# processed on NEMO so far
set.seed(42)
library(tidyverse)
library(SummarizedExperiment)
library(readxl)
library(glmnet)
library(survival)
setwd("/camp/home/hungm/scratch/hungm/matthew/MH_Hackathon/2025_CRUK/data/TCGA2/0_raw")

In [9]:
# read gene expression
gex <- read.csv("../1_processed/TCGA_lung_gex.csv", row.names = 1)
colnames(gex) <- gsub("\\.", "-", colnames(gex))
gex <- log1p(gex)
gex[1:5, 1:5]

,TCGA-05-4244,TCGA-05-4249,TCGA-05-4250,TCGA-05-4382,TCGA-05-4384
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A1BG,3.296955,4.7969056,3.948542,4.989779,4.854893
A1CF,0.000000,0.2791457,0.000000,0.000000,0.000000
A2BP1,1.009927,0.9592736,0.000000,0.000000,0.000000
A2LD1,4.916339,4.5005083,5.024755,4.727998,4.483847
A2M,9.194799,10.1547792,9.737704,10.057380,10.785510


In [10]:
# read survival data
surv <- read.csv("../1_processed/TCGA_lung_survival.csv", row.names = 1)
colnames(surv) <- gsub("\\.", "_", colnames(surv))

# remove time < 0 or NA
surv <- surv %>% filter(!is.na(PFI_time_1))
surv$PFI_time_1 <- as.numeric(surv$PFI_time_1)/365.25
surv$PFI_1 <- as.numeric(surv$PFI_1)

# set max PFI to 10 years
surv$PFI_1 <- ifelse(surv$PFI_time_1 > 10, 0, surv$PFI_1)
surv$PFI_time_1 <- ifelse(surv$PFI_time_1 > 10, 10, surv$PFI_time_1)
surv <- surv[surv$PFI_time_1 > 0, ]

# create survival object
surv$PFI <- Surv(surv$PFI_time_1, surv$PFI_1)
head(surv)

,type,PFI_1,PFI_time_1,PFI_2,PFI_time_2,PFS,PFS_time,DSS_cr,DSS_time_cr,DFI_cr,DFI_time_cr,PFI_cr,PFI_time_cr,PFI_1_cr,PFI_time_1_cr,PFI_2_cr,PFI_time_2_cr,PFI
,<chr>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<Surv>
TCGA-05-4249,LUAD,0,4.1697467,0,1523,0,1523,0,1523,NA,NA,0,1523,0,1523,0,1523,"4.1697467, 0"
TCGA-05-4250,LUAD,0,0.3312799,0,121,1,121,NA,121,NA,NA,2,121,2,121,2,121,"0.3312799, 0"
TCGA-05-4382,LUAD,1,0.9144422,1,334,1,334,0,607,1,334,1,334,1,334,1,334,"0.9144422, 1"
TCGA-05-4384,LUAD,1,0.5010267,1,183,1,183,0,426,NA,NA,1,183,1,183,1,183,"0.5010267, 1"
TCGA-05-4389,LUAD,0,3.7481177,0,1369,0,1369,0,1369,NA,NA,0,1369,0,1369,0,1369,"3.7481177, 0"
TCGA-05-4390,LUAD,1,1.0814511,NA,NA,1,395,0,1126,NA,NA,1,395,1,395,NA,NA,"1.0814511, 1"


In [11]:
# merge expression with survival data
surv <- cbind(surv, t(gex)[rownames(surv),])
surv[1:5, 15:20]

,PFI_time_1_cr,PFI_2_cr,PFI_time_2_cr,PFI,A1BG,A1CF
,<int>,<int>,<int>,<Surv>,<dbl>,<dbl>
TCGA-05-4249,1523,0,1523,"4.1697467, 0",4.796906,0.2791457
TCGA-05-4250,121,2,121,"0.3312799, 0",3.948542,0.0000000
TCGA-05-4382,334,1,334,"0.9144422, 1",4.989779,0.0000000
TCGA-05-4384,183,1,183,"0.5010267, 1",4.854893,0.0000000
TCGA-05-4389,1369,0,1369,"3.7481177, 0",4.221578,3.6174264


## Univariate Analysis

### All LUNG
Removed patients without outcome/survival time = 0, then do univariate cox regression to filter prognostic genes (4184, p < 0.05). Performed lasso-cox regression (a = 1, nfolds = 5) resulting in 151 prognostic genes.


In [12]:
library(survival)
library(doParallel)
library(foreach)

# Set up parallel backend
num_cores <- 30  # Use all available cores minus one
cl <- makeCluster(num_cores)
registerDoParallel(cl)

# Initialize parallel processing
univariate.genes <- foreach(gene = rownames(gex), .combine = rbind, .packages = c("survival")) %dopar% {
    # Create formula dynamically
    formula <- as.formula(paste0("PFI ~ `", gene, "`"))
    
    # Fit Cox model
    cox <- summary(coxph(formula, data = surv))
    
    # Extract hazard ratio (HR) and p-value
    hr <- as.data.frame(cox$coef)[[2]][1]
    pvalue <- as.data.frame(cox$coef)[[5]][1]
    
    # Return results as a data frame
    data.frame(gene = gene, hr = hr, pvalue = pvalue)
}

# Stop parallel backend
stopCluster(cl)

# View results
head(univariate.genes)

,gene,hr,pvalue
,<chr>,<dbl>,<dbl>
1,A1BG,0.9704959,0.618871566
2,A1CF,1.0644249,0.374985739
3,A2BP1,0.9554042,0.353871226
4,A2LD1,1.2651549,0.051995061
5,A2M,1.0336125,0.521458510
6,A2ML1,0.9438323,0.002350159


In [13]:
# filter genes with univariate-cox < 0.05
uni <- univariate.genes$gene[which(univariate.genes$pvalue < 0.05)]
length(uni)
uni

[1] 4184

[1] "A2ML1"          "A4GALT"         "AADAC"          "AADACL2"       
   [5] "ABCA13"         "ABCA5"          "ABCC1"          "ABCC3"         
   [9] "ABCC5"          "ABCC6"          "ABCC6P1"        "ABCD1"         
  [13] "ABCF3"          "ABCG5"          "ABHD10"         "ABHD6"         
  [17] "ABI1"           "ABI2"           "ABL1"           "ABL2"          
  [21] "ABLIM3"         "ABP1"           "ACAD11"         "ACAD9"         
  [25] "ACAP1"          "ACAP2"          "ACBD5"          "ACCN2"         
  [29] "ACCN4"          "ACCSL"          "ACO1"           "ACO2"          
  [33] "ACOT2"          "ACOT4"          "ACOT6"          "ACOT9"         
  [37] "ACOX2"          "ACP2"           "ACPL2"          "ACPP"          
  [41] "ACSBG1"         "ACSF2"          "ACSF3"          "ACSL3"         
  [45] "ACSL4"          "ACSL5"          "ACSS1"          "ACTL6A"        
  [49] "ACTN1"          "ACVR1C"         "ACY3"           "ADAD2"         
  [53] "ADAL"           "ADAM20"         "ADAM23"         "ADAM6"         
  [57] "ADAM8"          "ADAMTS17"       "ADAMTS20"       "ADAMTS6"       
  [61] "ADAMTS7"        "ADAMTS9"        "ADAP1"          "ADAR"          
  [65] "ADAT2"          "ADCY10"         "ADCY2"          "ADD1"          
  [69] "ADD3"           "ADH1C"          "ADH5"           "ADH7"          
  [73] "ADI1"           "ADORA1"         "ADPGK"          "ADRA1B"        
  [77] "ADRA2B"         "ADSL"           "ADSS"           "ADSSL1"        
  [81] "AEBP2"          "AFF1"           "AFF2"           "AFF4"          
  [85] "AGA"            "AGAP11"         "AGFG1"          "AGGF1"         
  [89] "AGK"            "AGL"            "AGPAT1"         "AGPAT2"        
  [93] "AGPAT6"         "AGPS"           "AGR2"           "AGRN"          
  [97] "AGT"            "AGXT2L1"        "AHCYL2"         "AHDC1"         
 [101] "AHRR"           "AIDA"           "AIFM2"          "AIFM3"         
 [105] "AIG1"           "AKAP12"         "AKAP2"          "AKAP3"         
 [109] "AKAP6"          "AKAP7"          "AKAP8"          "AKR1A1"        
 [113] "AKR1B10"        "AKR1C1"         "AKR1C2"         "ALDH16A1"      
 [117] "ALDH1A1"        "ALDH3A1"        "ALDH3B1"        "ALDH3B2"       
 [121] "ALDH4A1"        "ALDH7A1"        "ALDH8A1"        "ALDOC"         
 [125] "ALG1L"          "ALG3"           "ALK"            "ALKBH2"        
 [129] "ALKBH3"         "ALKBH7"         "ALOX12"         "ALOX15"        
 [133] "ALPI"           "ALPK3"          "ALPP"           "ALPPL2"        
 [137] "ALS2"           "ALS2CR4"        "AMD1"           "AMDHD1"        
 [141] "AMIGO2"         "AMPD1"          "AMTN"           "ANAPC13"       
 [145] "ANAPC5"         "ANAPC7"         "ANG"            "ANGPTL4"       
 [149] "ANGPTL6"        "ANKFY1"         "ANKK1"          "ANKRD13B"      
 [153] "ANKRD16"        "ANKRD2"         "ANKRD20A3"      "ANKRD20B"      
 [157] "ANKRD28"        "ANKRD32"        "ANKRD49"        "ANKRD54"       
 [161] "ANKRD9"         "ANO10"          "ANO3"           "ANO5"          
 [165] "ANO6"           "ANO7"           "ANP32C"         "ANP32E"        
 [169] "ANTXR2"         "ANXA11"         "ANXA2P1"        "ANXA5"         
 [173] "ANXA8"          "ANXA8L2"        "ANXA9"          "AOAH"          
 [177] "AOX2P"          "AP1B1"          "AP1S3"          "AP2A2"         
 [181] "AP2B1"          "AP2M1"          "AP3B1"          "AP3B2"         
 [185] "AP3S1"          "AP4S1"          "APAF1"          "APBB1"         
 [189] "APBB2"          "APCDD1"         "APCDD1L"        "APEX2"         
 [193] "APOB"           "APOBEC1"        "APOBEC3A"       "APOBEC3B"      
 [197] "APOBEC3C"       "APOBEC3D"       "APOBEC3F"       "APOC3"         
 [201] "APPL2"          "AQP10"          "AQP7P1"         "AQP7P3"        
 [205] "AREG"           "ARF5"           "ARFIP1"         "ARHGAP12"      
 [209] "ARHGAP19"       "ARHGAP23"       "ARHGAP24"       "ARHGAP25"      
 [213] "ARHGAP26"       "ARH

In [14]:
# do lasso cox regression to shrink important genes
X <- t(gex[uni, rownames(surv)])
y <- Surv(surv$PFI_time_1, surv$PFI_1)
cv.lasso_cox <- cv.glmnet(X, y, family = "cox", alpha = 1, nfolds = 5, type.measure="C")

# filter important genes
coef_lasso <- coef(cv.lasso_cox, s = "lambda.min")
lasso_genes <- rownames(coef_lasso)[which(coef_lasso[,1] > 0)]
print(lasso_genes)

  [1] "ABCD1"        "ABCF3"        "ADAMTS9"      "ANKRD32"      "AP3B1"       
  [6] "AP3S1"        "AP4S1"        "AQP10"        "AQP7P1"       "AQP7P3"      
 [11] "ATG10"        "C11orf36"     "C14orf53"     "C17orf85"     "C20orf141"   
 [16] "C2orf54"      "C2orf82"      "C5orf36"      "C5orf43"      "C6orf195"    
 [21] "C8orf74"      "CDHR5"        "CLRN2"        "CRIM1"        "CT47A10"     
 [26] "CTNNA2"       "CTNS"         "DEFB123"      "DEFB132"      "DHRS7C"      
 [31] "DKK1"         "DLEU7"        "DNAJA1"       "DSCAM"        "DYSFIP1"     
 [36] "EBF2"         "EML6"         "ENOSF1"       "ERLIN1"       "FADD"        
 [41] "FAM9A"        "FER"          "FGF4"         "FKBP3"        "FLJ40434"    
 [46] "FLJ44054"     "FLJ46111"     "FUBP1"        "GLDC"         "GRB10"       
 [51] "GYPB"         "H19"          "H6PD"         "HNRNPA0"      "HOXB4"       
 [56] "HPX"          "HSD3B2"       "HSF2BP"       "HSPA1L"       "IGSF5"       
 [61] "INCA1"        "IRS1" 

In [16]:
write.table(lasso_genes, "../2_metastatic_genes/TCGA_lung_univariate_lasso.txt", col.names = F, row.names = F, quote = F, sep = "\t")

### LUSC
LUSC patients only, univariate cox regression to filter	prognostic genes (1917, p < 0.05). Performed lasso-cox regression (a = 1, nfolds = 5) resulting	in 97 prognostic genes.

In [17]:
LUSC <- surv %>% 
    filter(type == "LUSC")
dim(LUSC)

[1]   495 20519

In [18]:
library(survival)
library(doParallel)
library(foreach)

# Set up parallel backend
num_cores <- 30  # Use all available cores minus one
cl <- makeCluster(num_cores)
registerDoParallel(cl)

# Initialize parallel processing
univariate.genes <- foreach(gene = rownames(gex), .combine = rbind, .packages = c("survival")) %dopar% {
    # Create formula dynamically
    formula <- as.formula(paste0("PFI ~ `", gene, "`"))
    
    # Fit Cox model
    cox <- summary(coxph(formula, data = LUSC))
    
    # Extract hazard ratio (HR) and p-value
    hr <- as.data.frame(cox$coef)[[2]][1]
    pvalue <- as.data.frame(cox$coef)[[5]][1]
    
    # Return results as a data frame
    data.frame(gene = gene, hr = hr, pvalue = pvalue)
}

# Stop parallel backend
stopCluster(cl)

# View results
head(univariate.genes)

,gene,hr,pvalue
,<chr>,<dbl>,<dbl>
1,A1BG,0.9488596,0.60050772
2,A1CF,0.8108010,0.57331082
3,A2BP1,0.9169648,0.32849577
4,A2LD1,0.8005384,0.24910305
5,A2M,1.0612524,0.46178884
6,A2ML1,0.9281850,0.04752442


In [19]:
# filter genes with univariate-cox < 0.05
uni <- univariate.genes$gene[which(univariate.genes$pvalue < 0.05)]
length(uni)
uni

[1] 1917

[1] "A2ML1"        "AADACL2"      "ABCA13"       "ABCA2"       
   [5] "ABCA3"        "ABCB10"       "ABCC5"        "ABCD1"       
   [9] "ABCF1"        "ABCG1"        "ABHD10"       "ABI2"        
  [13] "ABL1"         "ABL2"         "ABLIM3"       "ABP1"        
  [17] "ABR"          "ABTB2"        "ACN9"         "ACO2"        
  [21] "ACOT11"       "ACPL2"        "ACTG1"        "ACTN1"       
  [25] "ADAD2"        "ADAM21"       "ADAM21P1"     "ADAMTS15"    
  [29] "ADAMTS17"     "ADAMTSL4"     "ADAT1"        "ADCY2"       
  [33] "ADCY7"        "ADCYAP1R1"    "ADD1"         "ADH1C"       
  [37] "ADH7"         "ADI1"         "ADRA2C"       "ADSS"        
  [41] "ADSSL1"       "AES"          "AFF1"         "AGAP1"       
  [45] "AGBL1"        "AGL"          "AGPAT2"       "AGPHD1"      
  [49] "AGPS"         "AIFM2"        "AIMP2"        "AJAP1"       
  [53] "AK1"          "AKAP12"       "AKR1A1"       "AKR1D1"      
  [57] "ALDH3B1"      "ALDH3B2"      "ALDH7A1"      "ALG14"       
  [61] "ALKBH2"       "ALKBH3"       "ALKBH7"       "ALPI"        
  [65] "ALPK3"        "ALPP"         "ALPPL2"       "ALS2"        
  [69] "AMBP"         "ANAPC13"      "ANAPC2"       "ANAPC5"      
  [73] "ANGEL2"       "ANKFY1"       "ANKRD16"      "ANKRD27"     
  [77] "ANKRD46"      "ANKRD9"       "ANKS1B"       "ANO5"        
  [81] "ANTXRL"       "AP1S3"        "AP2A2"        "AP2B1"       
  [85] "AP2M1"        "AP3B2"        "AP4B1"        "APBB2"       
  [89] "APCDD1L"      "APOBEC3C"     "APOD"         "AQP7P1"      
  [93] "AQP7P3"       "ARC"          "AREG"         "ARHGAP24"    
  [97] "ARHGAP39"     "ARHGEF10"     "ARHGEF17"     "ARHGEF2"     
 [101] "ARHGEF38"     "ARHGEF4"      "ARMC9"        "ARPC5L"      
 [105] "ARSG"         "ARV1"         "AS3MT"        "ASAP2"       
 [109] "ASB1"         "ASB6"         "ASH2L"        "ASNA1"       
 [113] "ASTE1"        "ASXL1"        "ASXL2"        "ATAD1"       
 [117] "ATG4D"        "ATP10A"       "ATP11A"       "ATP13A5"     
 [121] "ATP1B3"       "ATP5F1"       "ATP5G1"       "ATP5G2"      
 [125] "ATP5G3"       "ATP5L"        "ATP5O"        "ATP6AP1"     
 [129] "ATP6V1E1"     "ATP7B"        "ATP8A1"       "ATP9A"       
 [133] "ATPIF1"       "ATXN1L"       "ATXN2"        "B3GALT5"     
 [137] "B3GNT3"       "B3GNT7"       "B4GALT5"      "BAG4"        
 [141] "BAHCC1"       "BAI3"         "BAP1"         "BASE"        
 [145] "BAT2"         "BAT2L1"       "BAZ2A"        "BAZ2B"       
 [149] "BCAR3"        "BCL11A"       "BCL2L2"       "BCORL1"      
 [153] "BDH1"         "BDP1"         "BHLHE40"      "BHMT2"       
 [157] "BICD1"        "BID"          "BIRC6"        "BLK"         
 [161] "BMI1"         "BMP7"         "BMPR2"        "BNC1"        
 [165] "BOD1L"        "BOLA1"        "BPNT1"        "BRF2"        
 [169] "BRIX1"        "BRPF1"        "BRPF3"        "BRSK2"       
 [173] "BSPRY"        "BTBD2"        "BTBD3"        "BTBD8"       
 [177] "BTBD9"        "BTG1"         "BTLA"         "BTN1A1"      
 [181] "BTN2A1"       "BZW2"         "C10orf41"     "C10orf99"    
 [185] "C11orf1"      "C11orf20"     "C11orf36"     "C11orf71"    
 [189] "C11orf86"     "C11orf9"      "C12orf51"     "C12orf54"    
 [193] "C13orf29"     "C13orf37"     "C14orf118"    "C14orf135"   
 [197] "C14orf162"    "C14orf167"    "C14orf174"    "C14orf21"    
 [201] "C14orf23"     "C14orf53"     "C15orf21"     "C15orf41"    
 [205] "C15orf55"     "C15orf61"     "C16orf63"     "C17orf50"    
 [209] "C17orf55"     "C17orf57"     "C17orf63"     "C17orf68"    
 [213] "C17orf76"     "C17orf85"     "C18orf26"     "C18orf55"    
 [217] "C19orf21"     "C19orf24"     "C19orf43"     "C19orf55"    
 [221] "C19orf56"     "C19orf63"     "C19orf70"     "C1orf101"    
 [225] "C1orf110"     "C1orf124"     "C1orf125"     "C1orf131"    
 [229] "C1orf186"     "C1orf187"     "C1orf21"      "C1orf51"     
 [233] "C1orf52"      "C20orf112"    "C20orf141"    "C20orf194"   
 [237] "C21orf119"    "C21orf63"     "C21orf84"   

In [20]:
# do lasso cox regression to shrink important genes
X <- t(gex[uni, rownames(LUSC)])
y <- Surv(LUSC$PFI_time_1, LUSC$PFI_1)
cv.lasso_cox <- cv.glmnet(X, y, family = "cox", alpha = 1, nfolds = 5, type.measure="C")

# filter important genes
coef_lasso <- coef(cv.lasso_cox, s = "lambda.min")
lasso_genes <- rownames(coef_lasso)[which(coef_lasso[,1] > 0)]
print(lasso_genes)

 [1] "ABCG1"        "ADRA2C"       "ANTXRL"       "AP1S3"        "ATP6AP1"     
 [6] "B3GALT5"      "BCL2L2"       "BICD1"        "C14orf135"    "C2orf70"     
[11] "C8orf74"      "C9orf131"     "CACNA1F"      "CCDC144B"     "CDH4"        
[16] "CLDN22"       "CLPS"         "CTSL3"        "CTTN"         "CXorf56"     
[21] "DCD"          "DEFB123"      "DLEU7"        "DNAL1"        "DSG2"        
[26] "DUSP15"       "FLJ35776"     "FLJ40434"     "FLOT1"        "GALNTL5"     
[31] "GPR78"        "GSC"          "HCRTR1"       "HOXD12"       "IBTK"        
[36] "IKBKAP"       "IL3"          "ITGB1BP3"     "KLK8"         "KSR2"        
[41] "LOC100133545" "LOC127841"    "LOC360030"    "LOC55908"     "LOC728875"   
[46] "LRRC20"       "MAGEB2"       "MARK3"        "MBOAT2"       "MYEOV"       
[51] "NKX2-2"       "NPLOC4"       "NR6A1"        "OR2W3"        "OR56A3"      
[56] "PALM3"        "PCDHA3"       "PCDHGB4"      "PDCL2"        "PDE10A"      
[61] "PITPNA"       "POMT1"        "PTPR

In [21]:
write.table(lasso_genes, "../2_metastatic_genes/TCGA_LUSC_univariate_lasso.txt", col.names = F, row.names = F, quote = F, sep = "\t")

### LUAD
LUAD patients only, univariate cox regression to filter	prognostic genes (3654, p < 0.05). Performed lasso-cox regression (a = 1, nfolds = 5) resulting in 110 prognostic genes.

In [22]:
LUAD <- surv %>% 
    filter(type == "LUAD")
dim(LUAD)

[1]   503 20519

In [23]:
library(survival)
library(doParallel)
library(foreach)

# Set up parallel backend
num_cores <- 30  # Use all available cores minus one
cl <- makeCluster(num_cores)
registerDoParallel(cl)

# Initialize parallel processing
univariate.genes <- foreach(gene = rownames(gex), .combine = rbind, .packages = c("survival")) %dopar% {
    # Create formula dynamically
    formula <- as.formula(paste0("PFI ~ `", gene, "`"))
    
    # Fit Cox model
    cox <- summary(coxph(formula, data = LUAD))
    
    # Extract hazard ratio (HR) and p-value
    hr <- as.data.frame(cox$coef)[[2]][1]
    pvalue <- as.data.frame(cox$coef)[[5]][1]
    
    # Return results as a data frame
    data.frame(gene = gene, hr = hr, pvalue = pvalue)
}

# Stop parallel backend
stopCluster(cl)

# View results
head(univariate.genes)

,gene,hr,pvalue
,<chr>,<dbl>,<dbl>
1,A1BG,0.9185667,0.257531683
2,A1CF,1.0252673,0.731421619
3,A2BP1,0.9678568,0.577254467
4,A2LD1,1.5394511,0.005534639
5,A2M,0.8168842,0.012737910
6,A2ML1,1.1104490,0.007959814


In [24]:
# filter genes with univariate-cox < 0.05
uni <- univariate.genes$gene[which(univariate.genes$pvalue < 0.05)]
length(uni)
uni

[1] 3654

[1] "A2LD1"           "A2M"             "A2ML1"           "ABAT"           
   [5] "ABCA10"          "ABCA13"          "ABCA3"           "ABCA8"          
   [9] "ABCC11"          "ABCC12"          "ABCC4"           "ABCC6"          
  [13] "ABCC6P1"         "ABCC6P2"         "ABCC8"           "ABHD14B"        
  [17] "ABHD6"           "ABI3"            "ABI3BP"          "ABR"            
  [21] "ABTB1"           "ACAD8"           "ACADS"           "ACADSB"         
  [25] "ACAN"            "ACAP1"           "ACAT1"           "ACBD4"          
  [29] "ACCN4"           "ACCS"            "ACCSL"           "ACE"            
  [33] "ACHE"            "ACOT12"          "ACOXL"           "ACP1"           
  [37] "ACP5"            "ACPL2"           "ACPP"            "ACRBP"          
  [41] "ACRV1"           "ACSBG1"          "ACSF3"           "ACSL3"          
  [45] "ACSL5"           "ACSM3"           "ACSM5"           "ACSS1"          
  [49] "ACTB"            "ACTL6A"          "ACTN1"           "ACTN4"          
  [53] "ACTR2"           "ACTR3"           "ACVRL1"          "ADA"            
  [57] "ADAM10"          "ADAM12"          "ADAM5P"          "ADAM6"          
  [61] "ADAMTS17"        "ADAMTS20"        "ADAMTS4"         "ADAMTS6"        
  [65] "ADAMTS8"         "ADAMTSL2"        "ADC"             "ADCY4"          
  [69] "ADCY6"           "ADCY9"           "ADCYAP1R1"       "ADH1A"          
  [73] "ADH1B"           "ADH7"            "ADHFE1"          "ADIPOQ"         
  [77] "ADIPOR2"         "ADM"             "ADORA2A"         "ADRB2"          
  [81] "AGAP11"          "AGAP2"           "AGAP3"           "AGAP4"          
  [85] "AGAP5"           "AGAP7"           "AGBL2"           "AGER"           
  [89] "AGFG1"           "AGFG2"           "AGMAT"           "AGPAT5"         
  [93] "AGPS"            "AHNAK2"          "AHRR"            "AHSA1"          
  [97] "AHSG"            "AIFM3"           "AIMP2"           "AK1"            
 [101] "AK2"             "AKAP1"           "AKAP12"          "AKAP13"         
 [105] "AKAP14"          "AKAP3"           "AKD1"            "AKNA"           
 [109] "AKT2"            "ALDH2"           "ALDH4A1"         "ALDOA"          
 [113] "ALG10"           "ALG3"            "ALG8"            "ALK"            
 [117] "ALOX15"          "ALOX5"           "ALOXE3"          "ALPI"           
 [121] "ALPL"            "ALS2CR12"        "ALX1"            "AMBP"           
 [125] "AMDHD1"          "AMICA1"          "AMPD1"           "AMT"            
 [129] "ANGPTL4"         "ANGPTL5"         "ANGPTL6"         "ANK2"           
 [133] "ANKHD1"          "ANKK1"           "ANKMY1"          "ANKRD13A"       
 [137] "ANKRD20A3"       "ANKRD32"         "ANKRD44"         "ANKRD55"        
 [141] "ANKRD57"         "ANKRD6"          "ANLN"            "ANO10"          
 [145] "ANO2"            "ANO9"            "ANP32A"          "ANP32B"         
 [149] "ANP32E"          "ANXA2P1"         "AOX2P"           "AP1S3"          
 [153] "AP2A1"           "AP2M1"           "AP2S1"           "AP3B1"          
 [157] "AP3S1"           "APCDD1L"         "APLP2"           "APOB48R"        
 [161] "APOBEC1"         "APOBEC3A"        "APOBEC3H"        "APOBEC4"        
 [165] "APOC4"           "APOH"            "AQP3"            "AQP4"           
 [169] "AQP5"            "AQP7"            "ARAP1"           "ARFGAP2"        
 [173] "ARHGAP11A"       "ARHGAP11B"       "ARHGAP15"        "ARHGAP24"       
 [177] "ARHGAP25"        "ARHGAP27"        "ARHGAP30"        "ARHGAP9"        
 [181] "ARHGDIB"         "ARHGEF15"        "ARHGEF17"        "ARHGEF2"        
 [185] "ARHGEF3"         "ARHGEF37"        "ARHGEF38"        "ARHGEF6"        
 [189] "ARID3A"          "ARIH2"           "ARL6IP1"         "ARL6IP6"        
 [193] "ARMC5"           "ARNTL2"          "ARPM1"           "ARRB1"          
 [197] "ARRB2"           "ARVCF"           "ASAH2"           "ASAP3"          
 [201] "ASB2"            "ASF1B"           "ASPA"    

In [25]:
# do lasso cox regression to shrink important genes
X <- t(gex[uni, rownames(LUAD)])
y <- Surv(LUAD$PFI_time_1, LUAD$PFI_1)
cv.lasso_cox <- cv.glmnet(X, y, family = "cox", alpha = 1, nfolds = 5, type.measure="C")

# filter important genes
coef_lasso <- coef(cv.lasso_cox, s = "lambda.min")
lasso_genes <- rownames(coef_lasso)[which(coef_lasso[,1] > 0)]
print(lasso_genes)

  [1] "ACOT12"       "ACOXL"        "ADAM5P"       "ARPM1"        "ATG10"       
  [6] "B3GALTL"      "BANP"         "C14orf145"    "C18orf2"      "C1orf94"     
 [11] "C6orf195"     "C7orf28A"     "CCDC126"      "CRCT1"        "CUL4A"       
 [16] "DAB1"         "DAOA"         "DGKK"         "DHRS7C"       "DISC2"       
 [21] "DKK1"         "DLX1"         "DNAJC1"       "EFNB2"        "ELOVL7"      
 [26] "FANCL"        "FGF6"         "FNDC4"        "FTSJ2"        "FUNDC2P2"    
 [31] "GAL3ST3"      "GCKR"         "GDPD2"        "GLRA4"        "GNGT1"       
 [36] "GOPC"         "GPX5"         "GRAMD1B"      "HGSNAT"       "HIST1H2AL"   
 [41] "HIST1H3I"     "HPX"          "HSPA6"        "INSL4"        "KCNA7"       
 [46] "KCNK2"        "KRT34"        "LDLRAD3"      "LINGO1"       "LIPK"        
 [51] "LOC100192426" "LOC148189"    "LOC653544"    "LOC728410"    "LRRC36"      
 [56] "LRRIQ4"       "MGC4473"      "MMP8"         "NCSTN"        "NLRP10"      
 [61] "NLRP13"       "NR5A1"

In [26]:
write.table(lasso_genes, "../2_metastatic_genes/TCGA_LUAD_univariate_lasso.txt", col.names = F, row.names = F, quote = F, sep = "\t")